<a href="https://colab.research.google.com/github/cs20m016/cs6910-Assignment-3/blob/main/Q2_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qq
import wandb

In [ ]:
wandb.login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
from keras.utils.vis_utils import plot_model

In [ ]:
def Recurrent_Neural_Network(epoch, dropout, number_of_filters, input_embedding_size, number_of_encoder_layers, number_of_decoder_layers, cell_type, learning_rate, optimiser_fn, activation_fn):
  batch_size = 64  # Batch size for training.
  epochs = 4  # Number of epochs to train for.
  latent_dim = 256  # Latent dimensionality of the encoding space.
  num_samples = 10000  # Number of samples to train on.
  # Path to the data txt file on disk.
  data_path = "dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
  val_path = "dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
  test_path = "dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"
  input_texts = []
  target_texts = []
  input_characters = set()
  target_characters = set()
  with open(data_path, "r", encoding="utf-8") as f:
      lines = f.read().split("\n")
  for line in lines[: len(lines) - 1]:
      target_text, input_text, _ = line.split("\t")
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      target_text = "\t" + target_text + "\n"
      input_texts.append(input_text)
      target_texts.append(target_text)
      for char in input_text:
          if char not in input_characters:
              input_characters.add(char)
      for char in target_text:
          if char not in target_characters:
              target_characters.add(char)

  input_characters.add(" ")
  target_characters.add(" ")
  input_characters = sorted(list(input_characters))
  target_characters = sorted(list(target_characters))
  num_encoder_tokens = len(input_characters)
  num_decoder_tokens = len(target_characters)
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])

  val_input_texts = []
  val_target_texts = []

  with open(val_path, "r", encoding="utf-8") as f:
      lines = f.read().split("\n")
  for line in lines[: len(lines) - 1]:
      target_text, input_text, _ = line.split("\t")
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      target_text = "\t" + target_text + "\n"
      val_input_texts.append(input_text)
      val_target_texts.append(target_text)

  test_input_texts = []
  test_target_texts = []

  with open(test_path, "r", encoding="utf-8") as f:
      lines = f.read().split("\n")
  for line in lines[: len(lines) - 1]:
      target_text, input_text, _ = line.split("\t")
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      test_input_texts.append(input_text)
      test_target_texts.append(target_text)

  input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
  target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
  decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
  decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
  #print(np.shape(encoder_input_data),np.shape(decoder_input_data),np.shape(decoder_target_data))

  val_encoder_input_data = np.zeros((len(val_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
  val_decoder_input_data = np.zeros((len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
  val_decoder_target_data = np.zeros((len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
  #print(np.shape(val_encoder_input_data),np.shape(val_decoder_input_data),np.shape(val_decoder_target_data))

  test_encoder_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

  for i, (val_input_text, val_target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(val_input_text):
        val_encoder_input_data[i, t, input_token_index[char]] = 1.0
    val_encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(val_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

    for i, (test_input_text, ) in enumerate(zip(test_input_texts, )):
      for t, char in enumerate(test_input_text):
          test_encoder_input_data[i, t, input_token_index[char]] = 1.0
      test_encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

  # Define an input sequence and process it.
  if cell_type=='LSTM':
    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]
    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

  elif cell_type=='RNN':
    encoder = keras.layers.SimpleRNN(latent_dim, return_state=True,dropout=dropout,recurrent_dropout=recc_dropout)
    encoder_outputs, state_h = encoder(encoder_inputs)
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    decoder_rnn = keras.layers.SimpleRNN(latent_dim, return_sequences=True,return_state=True,dropout=dropout,recurrent_dropout=recc_dropout)
    decoder_outputs,_ = decoder_rnn(decoder_inputs, initial_state=state_h)

  elif cell_type=='GRU':
    encoder = keras.layers.GRU(latent_dim, return_state=True,dropout=dropout,recurrent_dropout=recc_dropout)
    encoder_outputs, state_h = encoder(encoder_inputs)
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True,return_state=True,dropout=dropout,recurrent_dropout=recc_dropout)
    decoder_outputs,_ = decoder_gru(decoder_inputs, initial_state=state_h)
    
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)
  
  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  if optimiser_fn=='adam':
    opt = tf.keras.optimizers.Adam(learning_rate = learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
  elif optimiser_fn=='sgd':
    opt = tf.keras.optimizers.SGD(learning_rate = learning_rate, momentum=0.0, nesterov=False, name="SGD")
  elif optimiser_fn=='momentum':
    opt = tf.keras.optimizers.SGD(learning_rate = learning_rate, momentum=0.9, nesterov=False)

  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data))


In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'model_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epoch': {
            'values': [40,50,60]
        },
        'dropout':{
            'values': [0.1,0.2,0.3]
        },
        'input_embedding_size': {
            'values': [16,32,64,256]
        },
        'number_of_encoder_layers': {
            'values': [1,2,3,4]
        },
        'number_of_decoder_layers':{
            'values': [1,2,3,4]
        },
        'cell_type': {
            'values':['RNN','GRU','LSTM']
        },
        'learning_rate':{
            'values': [0.01,0.001,0.0001]
        },
        'optimizer_fn':{
            'values': ['adam','sgd','momentum']
        },
        'activation_fn':{
            'values': ['relu','tanh']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config,entity="dlassignment3_",project="Part-A")

In [ ]:
def train():
    config_defaults={
        'epoch':40,
        'dropout':0.2,
        'input_embedding_size':32,
        'number_of_encoder_layers':1,
        'number_of_decoder_layers':1,
        'cell_type':'RNN',
        'learning_rate':0.001,
        'optimiser_fn':'adam',
        'activation_fn':'relu',
        }
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    Recurrent_Neural_Network(config.epoch, config.dropout, config.input_embedding_size, config.number_of_encoder_layers,config.number_of_decoder_layers, config.cell_type, config.learning_rate, config.optimiser_fn, config.activation_fn)  
    return